In [68]:
!pip install shap lime transformers torch -q
!pip install lime --quiet

In [69]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [70]:
import shap
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import numpy as np
import matplotlib.pyplot as plt
from lime.lime_text import LimeTextExplainer
import os

# Load Fine-tuned Model and tokenizer

In [71]:
import os

# model_dir = "/content/xlm-roberta-amharic-ner"
# print(os.listdir(model_dir))
# Unzip to content folder
!unzip /content/drive/MyDrive/xlm-roberta-amharic-ner.zip -d /content/xlm-roberta-amharic-ner

Archive:  /content/drive/MyDrive/xlm-roberta-amharic-ner.zip
replace /content/xlm-roberta-amharic-ner/model.safetensors? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [72]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
model_dir = "/content/xlm-roberta-amharic-ner"

tokenizer = AutoTokenizer.from_pretrained(model_dir, local_files_only=True)
model = AutoModelForTokenClassification.from_pretrained(model_dir, local_files_only=True)

ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

Device set to use cpu


In [73]:
sample_texts = "CLASSY BRAND NFL Jersey Tshirt Price 2500br Shop Num 0907000777"


#Prepare model prediction function for SHAP

In [74]:
def predict_logits_for_shap(texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits  # [batch_size, seq_len, num_labels]
    # Aggregate per-token logits to a single vector (e.g., by mean pooling)
    return logits.mean(dim=1).numpy()  # [batch_size, num_labels]

    # SHAP requires a masker; for text tasks, we can use shap.maskers.Text
masker = shap.maskers.Text(tokenizer)

#Create SHAP explainer

In [75]:
# Use shap.Explainer
explainer = shap.Explainer(predict_logits_for_shap, masker)

#Compute SHAP values

In [76]:
shap_values = explainer([sample_texts])

In [77]:
shap.plots.text(shap_values[0])

In [80]:
!git clone https://github.com/tnsay/ethioMart-telegram-ner.git
!cp "/content/drive/MyDrive/Colab_Notebooks/Model_Interpretability_EthioMart.ipynb" ethioMart-telegram-ner/
#%cd ethioMart-telegram-ner


Cloning into 'ethioMart-telegram-ner'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 40 (delta 11), reused 27 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (40/40), 36.45 KiB | 2.43 MiB/s, done.
Resolving deltas: 100% (11/11), done.
